In [1]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [2]:
MAIN_FOLDER = main_folder("pandora")

create_database('blp_dataset')
create_table('products_tmp')

In [4]:
original_images = []

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\item")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())

            ## GET DESCRIPTION
            description = ""
            # find description main container
            raw_desc = page_content.find("pre")

            if raw_desc is not None:
                # remove html content, whitespaces and line breaks
                description = get_content(raw_desc)

            ## GET TITLE
            title = ""
            raw_title = page_content.find("th", attrs={"colspan": "2"})

            if raw_title is not None:
                for strong in raw_title.find_all("strong", recursive=False):
                    if strong is not None:
                        title = get_content(strong)

            ## GET SELLER
            seller = ""
            seller_a = page_content.find_all("a", href=lambda value: value and value.startswith("/profile/"))

            if seller_a is not None:
                for seller_raw in seller_a:
                    seller = get_content(seller_raw)

            if description != "" or title != "":
                # Insert product into database
                query = "INSERT INTO products_tmp (name, description, market_name, seller_name, illegal) VALUES (%s, %s, %s, %s, %s)"
                values = (title, description, "Pandora", seller, "t")

                cursor.execute(query, values)
                conn.commit()

                product_id = cursor.lastrowid

                ## GET IMAGE
                raw_name = page_content.find("img")['src']

                if raw_name == "img/no-image.png":
                    query = "UPDATE products SET has_image = %s WHERE id = %s"
                    cursor.execute(query, ("no_image", product_id))
                    conn.commit()
                else:
                    if raw_name in original_images:
                        query = "UPDATE products SET has_image = %s WHERE id = %s"
                        cursor.execute(query, ("repeated", product_id))
                        conn.commit()
                    else:
                        original_images.append(raw_name)

                        url_image = Path(f"{folder_image}\{raw_name}")

                        # copy images to new directory, renaming with product id
                        if url_image.exists():
                            dst_dir = Path(f"D:\images\pandora\{product_id}.jpg")
                            copyfile(url_image, dst_dir)
                        else:
                            query = "UPDATE products SET has_image = %s WHERE id = %s"
                            cursor.execute(query, ("no_image", product_id))
                            conn.commit()